In [1]:
# install necessary packages finta, gym, jupyterlab, matplotlib, mplfinance, numpy, pandas, quantstats, stable_baselines3, torch
# %pip install finta gym jupyterlab matplotlib mplfinance numpy pandas quantstats stable_baselines3 torch keras tensorflow gym-anytrading


In [20]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import deque
import datetime
import mplfinance 
import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from scipy.signal import lfilter

import numpy as np
import matplotlib.pyplot as plt

import gymnasium as gym
import gym_anytrading
from gym_anytrading.envs import ForexEnv, StocksEnv, Actions, Positions 

import sys
sys.path.append("C:/Users/WilliamFetzner/Documents/Trading/")
import fx # personal functions
sys.path.append("C:/Users/WilliamFetzner/Documents/Trading/FinRL-Meta/")
import importlib
from meta.env_fx_trading.env_fx import tgym

import finta
import polars as pl

# Parameters

In [3]:
# penalty amount
penalty_amount = 0.05

In [24]:
full_ohlc_15_min_df = (
        pl.scan_csv(f'C:/Users/WilliamFetzner/Documents/Trading/EURUSD_full_tickstory_data_15_min.csv')
        .with_columns([
            pl.col('').alias('datetime').str.to_datetime(format='%Y-%m-%d %H:%M:%S'),
        ]).with_columns([
            pl.col('datetime').dt.week().alias('week_number'),
            # weekday
            pl.col('datetime').dt.weekday().alias('weekday'),
            pl.lit('EURUSD').alias('symbol'),
        ]).filter(pl.col('datetime').dt.year() == 2024)
        .drop('')
    ).collect()  
full_ohlc_15_min_df

Open,High,Low,Close,datetime,week_number,weekday,symbol
f64,f64,f64,f64,datetime[μs],i8,i8,str
1.10427,1.10433,1.10423,1.10431,2024-01-02 00:00:00,1,2,"""EURUSD"""
1.1043,1.10434,1.1042,1.1043,2024-01-02 00:15:00,1,2,"""EURUSD"""
1.10431,1.10437,1.10431,1.10433,2024-01-02 00:30:00,1,2,"""EURUSD"""
1.10426,1.10447,1.10423,1.10438,2024-01-02 00:45:00,1,2,"""EURUSD"""
1.10446,1.10446,1.10383,1.10383,2024-01-02 01:00:00,1,2,"""EURUSD"""
…,…,…,…,…,…,…,…
1.09183,1.09189,1.09179,1.09187,2024-08-09 22:45:00,32,5,"""EURUSD"""
1.09186,1.09194,1.09167,1.09168,2024-08-09 23:00:00,32,5,"""EURUSD"""
1.09168,1.09179,1.09167,1.09175,2024-08-09 23:15:00,32,5,"""EURUSD"""


In [33]:
# save all of the dataframes to csv files
files = []
num_weeks = full_ohlc_15_min_df.select(pl.col('week_number').max()).item()
for i in range(2, num_weeks + 1):
    week_i = full_ohlc_15_min_df.filter(pl.col('week_number') == i)
    week_i = week_i.drop('week_number').to_pandas()
    week_i['datetime'] = pd.to_datetime(week_i['datetime'])
    path = 'C:/Users/WilliamFetzner/Documents/Trading/FinRL-Meta/meta/env_fx_trading/data/train_week' + str(i) + '.csv'
    week_i.to_csv(path)
    files.append(path)
files_train = files[:10]
files_train

['C:/Users/WilliamFetzner/Documents/Trading/FinRL-Meta/meta/env_fx_trading/data/train_week2.csv',
 'C:/Users/WilliamFetzner/Documents/Trading/FinRL-Meta/meta/env_fx_trading/data/train_week3.csv',
 'C:/Users/WilliamFetzner/Documents/Trading/FinRL-Meta/meta/env_fx_trading/data/train_week4.csv',
 'C:/Users/WilliamFetzner/Documents/Trading/FinRL-Meta/meta/env_fx_trading/data/train_week5.csv',
 'C:/Users/WilliamFetzner/Documents/Trading/FinRL-Meta/meta/env_fx_trading/data/train_week6.csv',
 'C:/Users/WilliamFetzner/Documents/Trading/FinRL-Meta/meta/env_fx_trading/data/train_week7.csv',
 'C:/Users/WilliamFetzner/Documents/Trading/FinRL-Meta/meta/env_fx_trading/data/train_week8.csv',
 'C:/Users/WilliamFetzner/Documents/Trading/FinRL-Meta/meta/env_fx_trading/data/train_week9.csv',
 'C:/Users/WilliamFetzner/Documents/Trading/FinRL-Meta/meta/env_fx_trading/data/train_week10.csv',
 'C:/Users/WilliamFetzner/Documents/Trading/FinRL-Meta/meta/env_fx_trading/data/train_week11.csv']

In [16]:
files_train = ['C:/Users/WilliamFetzner/Documents/Trading/FinRL-Meta/meta/env_fx_trading/data/train_week2.csv',]
#  'C:/Users/WilliamFetzner/Documents/Trading/FinRL-Meta/meta/env_fx_trading/data/train_week3.csv',
#  'C:/Users/WilliamFetzner/Documents/Trading/FinRL-Meta/meta/env_fx_trading/data/train_week4.csv',
#  'C:/Users/WilliamFetzner/Documents/Trading/FinRL-Meta/meta/env_fx_trading/data/train_week5.csv',
#  'C:/Users/WilliamFetzner/Documents/Trading/FinRL-Meta/meta/env_fx_trading/data/train_week6.csv',
#  'C:/Users/WilliamFetzner/Documents/Trading/FinRL-Meta/meta/env_fx_trading/data/train_week7.csv',
#  'C:/Users/WilliamFetzner/Documents/Trading/FinRL-Meta/meta/env_fx_trading/data/train_week8.csv',
#  'C:/Users/WilliamFetzner/Documents/Trading/FinRL-Meta/meta/env_fx_trading/data/train_week9.csv',
#  'C:/Users/WilliamFetzner/Documents/Trading/FinRL-Meta/meta/env_fx_trading/data/train_week10.csv',
#  'C:/Users/WilliamFetzner/Documents/Trading/FinRL-Meta/meta/env_fx_trading/data/train_week11.csv']

# Reinforcement Learning

In [30]:
df = pd.read_csv(
    files_train[0]) #2024-03-11 00:00:00

In [31]:
df

,Unnamed: 0,Open,High,Low,Close,datetime,weekday,symbol
0,0,1.09386,1.09442,1.09376,1.09434,2024-01-08 00:00:00,1,EURUSD
1,1,1.09434,1.09446,1.09418,1.09436,2024-01-08 00:15:00,1,EURUSD
2,2,1.09440,1.09444,1.09427,1.09439,2024-01-08 00:30:00,1,EURUSD
3,3,1.09440,1.09451,1.09439,1.09444,2024-01-08 00:45:00,1,EURUSD
4,4,1.09445,1.09456,1.09381,1.09391,2024-01-08 01:00:00,1,EURUSD
...,...,...,...,...,...,...,...,...
475,475,1.09533,1.09538,1.09504,1.09512,2024-01-12 22:45:00,5,EURUSD
476,476,1.09511,1.09526,1.09496,1.09497,2024-01-12 23:00:00,5,EURUSD
477,477,1.09499,1.09520,1.09489,1.09493,2024-01-12 23:15:00,5,EURUSD
478,478,1.09494,1.09495,1.09481,1.09489,2024-01-12 23:30:00,5,EURUSD


In [71]:
# Reload the module
importlib.reload(sys.modules['meta.env_fx_trading.env_fx'])

# Update the local reference to the reloaded module
tgym = sys.modules['meta.env_fx_trading.env_fx'].tgym

In [73]:
def train(env, agent, files, if_vix = True, **kwargs):
    learning_rate = kwargs.get('learning_rate', 2 ** -15)
    batch_size = kwargs.get('batch_size', 2 ** 11 )
    gamma = kwargs.get('gamma', 0.99)
    seed = kwargs.get('seed', 312)
    total_timesteps = kwargs.get('total_timesteps', 1e6)
    net_dimension = kwargs.get('net_dimension', 2**9)
    cwd = kwargs.get('cwd','./'+str(agent))

    # env_instance = map(env, [pd.read_csv(f) for f in files])
    if agent == 'ppo':
        from stable_baselines3 import PPO
        from stable_baselines3.common.vec_env import SubprocVecEnv, DummyVecEnv

        # env_train = [x.get_sb_env for x in env_instance ]
        vector_env = [lambda:env(df=pd.read_csv(f)) for f in files]
        env_train = SubprocVecEnv(vector_env)
        model = PPO("MlpPolicy", env_train, learning_rate=learning_rate, 
                    n_steps=2048, batch_size=batch_size, ent_coef=0.0, 
                    gamma=gamma, seed=seed)
        start_time = time.time()
        s = datetime.datetime.now()
        print(f'Training start: {s}')
        model.learn(total_timesteps=total_timesteps, tb_log_name = 'ppo')
        print('Training finished!')
        model_name = "./data/models/EURUSD_week_" + s.strftime('%Y%m%d%H%M%S')
        model.save(model_name)
        print(f'Trained model saved in {model_name}')
        print(f"training time: {(time.time() - start_time)}")

    else:
        raise ValueError('DRL library input is NOT supported. Please check.')
train(env=tgym, agent="ppo", files=files_train)

# Environment: 

## Keras Architecture
In this code snippet:

- We import necessary modules from TensorFlow and Keras.

- We define functions build_actor_network and build_critic_network to create the actor and critic networks using Keras' functional API.

- The actor network outputs a probability distribution over actions using softmax activation.

- The critic network estimates the state value function.

In [7]:
# Define the Keras model architecture for the actor network
def build_actor_network(state_dim, action_dim):
    inputs = layers.Input(shape=(state_dim,))
    dense1 = layers.Dense(64, activation='relu')(inputs)
    dense2 = layers.Dense(64, activation='relu')(dense1)
    outputs = layers.Dense(action_dim, activation='softmax')(dense2)  # Softmax for policy output
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

# Define the Keras model architecture for the critic network
def build_critic_network(state_dim):
    inputs = layers.Input(shape=(state_dim,))
    dense1 = layers.Dense(64, activation='relu')(inputs)
    dense2 = layers.Dense(64, activation='relu')(dense1)
    outputs = layers.Dense(1)(dense2)  # Linear output for value estimation
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

# Example usage:
state_dim = 4  # Example state dimension
action_dim = 2  # Example action dimension

actor_model = build_actor_network(state_dim, action_dim)
critic_model = build_critic_network(state_dim)

actor_model.summary()
critic_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,610 (18.01 KB)

 Trainable params: 4,610 (18.01 KB)

 Non-trainable params: 0 (0.00 B)

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,545 (17.75 KB)

 Trainable params: 4,545 (17.75 KB)

 Non-trainable params: 0 (0.00 B)

# PPO Loss Function
In this code snippet:

- The ppo_loss function defines a custom loss function that calculates the PPO loss based on the clipped surrogate objective.
- The loss function computes the PPO loss using the ratio of new probabilities to old probabilities and applies clipping to ensure stable updates.
- An entropy term is included in the loss function to encourage exploration.
- Finally, the actor model is compiled using the PPO loss function for training.


In [8]:
# PPO loss function
def ppo_loss(old_probs, advantages, clip_ratio):
    def loss(y_true, y_pred):
        new_probs = y_pred
        ratio = new_probs / old_probs

        clipped_ratio = tf.clip_by_value(ratio, 1 - clip_ratio, 1 + clip_ratio)
        surrogate_objective = tf.minimum(ratio * advantages, clipped_ratio * advantages)

        entropy = -tf.reduce_sum(new_probs * tf.math.log(new_probs + 1e-10), axis=-1)  # Entropy bonus

        ppo_loss = -tf.reduce_mean(surrogate_objective - 0.01 * entropy)  # Include entropy for exploration

        return ppo_loss

    return loss

# Example usage:
clip_ratio = 0.2  # Example clip ratio
advantages = np.random.rand(10)  # Example advantages array
old_probs = np.random.rand(10, 2)  # Example old probabilities array

# Compile the actor model with PPO loss function
actor_model.compile(optimizer=keras.optimizers.Adam(), loss=ppo_loss(old_probs, advantages, clip_ratio))

# Compute Advantages
In this custom compute_advantages function:

- The function calculates advantages using the Generalized Advantage Estimation (GAE) method, incorporating rewards, predicted values from the critic network, and discount factors.
- It penalizes holding trades over the weekend by adjusting advantages for transitions from Friday to Monday based on the 'day_of_week_transition' feature.
- You can adjust the penalty_amount based on your specific trading strategy and risk management considerations.

In [9]:
def compute_advantages(states, rewards, dones, values, gamma=0.99, lmbda=0.95):
    advantages = np.zeros_like(rewards)
    last_advantage = 0

    # Calculate advantages using Generalized Advantage Estimation (GAE)
    for t in reversed(range(len(rewards))):
        if dones[t]:
            next_value = 0
        else:
            next_value = values[t + 1]

        delta = rewards[t] + gamma * next_value - values[t]
        advantages[t] = last_advantage = delta + gamma * lmbda * last_advantage

    # Penalize holding trades over the weekend (e.g., Friday to Monday)
    for t in range(len(states) - 1):
        if states[t]['day_of_week_transition'] == 1 and states[t + 1]['day_of_week_transition'] == 0:
            advantages[t] -= (1-penalty_amount)*advantages[t]

    return advantages


# Training Loop

In this code snippet:
- The train_ppo function defines a training loop that collects experiences from interacting with the environment and performs PPO updates using the collected data.
- The compute_advantages function calculates advantages using rewards and values from the critic network.
- The PPO agent is initialized with actor and critic models.
- The training loop is executed for a specified number of epochs on an example environment (CartPole-v1).

In [10]:
# Define the training loop for PPO agent
def train_ppo(agent, env, epochs, batch_size):
    for epoch in range(epochs):
        states = []
        actions = []
        rewards = []
        old_probs = []
        dones = []

        state = env.reset()
        done = False

        while not done:
            action_probs = agent.actor_model.predict(np.array([state]))[0]
            action = np.random.choice(len(action_probs), p=action_probs)
            next_state, reward, done, _ = env.step(action)

            states.append(state)
            actions.append(action)
            rewards.append(reward)
            old_probs.append(action_probs)
            dones.append(done)

            state = next_state

        # Compute advantages using rewards and values from critic network
        advantages = compute_advantages(rewards, dones, agent.critic_model)

        # Perform PPO update using collected data
        actor_loss = agent.actor_model.train_on_batch(np.array(states), np.array(actions), sample_weight=advantages)
        critic_loss = agent.critic_model.train_on_batch(np.array(states), np.array(rewards))

        print(f"Epoch: {epoch + 1}, Actor Loss: {actor_loss}, Critic Loss: {critic_loss}")

# Example usage:
class PPOAgent:
    def __init__(self, actor_model, critic_model):
        self.actor_model = actor_model
        self.critic_model = critic_model

env = gym.make('forex-v0')
actor_model = build_actor_network(env.observation_space.shape[0], env.action_space.n)
critic_model = build_critic_network(env.observation_space.shape[0])
agent = PPOAgent(actor_model, critic_model)

train_ppo(agent, env, epochs=100, batch_size=32)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (1, 2) + inhomogeneous part.

In [14]:
env


<OrderEnforcing<PassiveEnvChecker<ForexEnv<forex-v0>>>>